In [3]:
# orwell9 tonalities
# inside 22edo
# using domblar + sc3

# TODO: search for a sound
# melodies, chords, chord progressions
# modulations


# 22edo also supports the orwell temperament, which uses the septimal subminor third as a generator
# (5 degrees) and forms mos scales with step patterns 3 2 3 2 3 2 3 2 2 and 1 2 2 1 2 2 1 2 2 1 2 2 2.
# Harmonically, orwell can be tuned more accurately in other temperaments, such as 31edo, 53edo and 84edo.
# But 22edo orwell has a leg-up on the others melodically, as the large and small steps of orwell[9]
# are easier to distinguish in 22.


# orwell[5]
# 5 5 5 5 2
# intervals:
# 1 = 2s/5s
#     classic minor 2nd
#     7/6, minor 3rd
# 2 = 7s/10s
#     5/4
#     15/11, 11/8
# 3 = 12s/15s
#     16/11, 22/15
#     8/5
# 4 = 17s/20s
#     17/10, 12/7
#     28/15, 15/8
# well, it's too complicated for me


# orwell[9]
# 3 2 3 2 2 3 2 3 2
# LsLssLsLs
# also mentioned sLsLsLsLs

# ? orwell[13]

# 3/1 divided into 7 equal steps
# 1 step = 7/6
# 3 steps = 8/5

# interesting write-up
# https://en.xen.wiki/w/Technical_Notes_for_Newbeams#Spun
# harmonic backbone of this piece is a sequence of essentially tempered dyadic chords in Orwell[13]

# equivalences
# 16/15 ~ 15/14
# 12/11 ~ 11/10
# 14/11 ~ 9/7
# 14/9 ~ 11/7
# 20/11 ~ 11/6
# 28/15 ~ 15/8

# https://en.xen.wiki/w/Dyadic_chord
# https://en.xen.wiki/w/Lumatone_mapping_for_orwell
# https://en.xen.wiki/w/Orwell_on_an_Isomorphic_Keyboard
# https://en.xen.wiki/w/Chords_of_orwell

In [6]:
import os
import sys
from pathlib import Path

# add path to domblar development code
sys.path.insert(0, str(Path(os.getcwd()).parent))

from domblar.domblar import Domblar

d = Domblar(
    context='python-sc3',
)

In [7]:
from domblar.edo import mos, intervals_in_edo

edo = 22
# FIXME: remove this scale, looks redundant
scale = list(range(edo))

scales = {}
scales['orwell9'] = mos(9, gen=5, edo=edo, down=0, shift=0) + [edo]

print(scales)

{'orwell9': [0, 3, 5, 8, 10, 13, 15, 18, 20, 22]}


In [ ]:
# TODO

from copy import copy
import random

from domblar.rhythm import get_grouped_rhythm, rhythm_to_notes
from domblar.transformations import shift

def set_synths():
    d.set_synth(0, 'chiptune_varsaw')
    d.set_synth(1, 'chiptune_varsaw')
set_synths()

cur_scale = scales['orwell9']
print(cur_scale)
print('steps -', intervals_in_edo(cur_scale, edo))
d.play(cur_scale, scale, edo, synth_idx=[0], dur=0.3)


In [ ]:
# music machines!

class FSM():
    def __init__(self):
        pass

song = FSM()
d.play(song)

# orwell9, 9 notes
# i, ii, iii, ...,
# but wait, what is the base chord shape?
# probably 0-2-4-6 and similar
# TODO: do we have minor/major? or what is the spectrum?

In [8]:
def find_nearest_note(scale, edo, cur, targets, prob=0.75, min_bound=None):
    residue = cur % len(scale)
    best_dist = None
    best_cand = None
    targets1 = list(set(targets))
    random.shuffle(targets1)
    for target in targets1:
        target_residue = target % len(scale)
        shifts = [-1, 0, 1]
        random.shuffle(shifts)
        for k in shifts:
            candidate = target - target_residue + residue + k * len(scale)
            if min_bound is not None and get_12edo(scale, edo, candidate) < min_bound:
                continue
            # FIXME: in theory we can get None for best_cand
            cur_dist = abs(candidate - target)
            if best_dist is None or (random.random() <= prob and cur_dist < best_dist):
                best_cand = candidate
                best_dist = cur_dist
        if best_cand is None:
            print('ERROR: best cand is None!')
            best_cand = candidate
    # todo: forbid voice crossings
    return best_cand

def smooth_chords(scale, edo, chords, start=0, min_bound=None, max_bound=None, prob=0.75):
    if start is None:
        prev_chord = [0]
    else:
        if type(start) is int:
            prev_chord = [start]
        else:
            prev_chord = start
    smoothed_chords = []
    for idx, chord in enumerate(chords):
        smoothed_chord = []
        for n in chord:
            smoothed_chord.append(find_nearest_note(scale, edo, n, prev_chord, min_bound=min_bound))
        
        # trying to place the root below other notes
        if (smoothed_chord[0] != min(smoothed_chord)) and (random.random() <= prob):
            if (random.random() <= 0.5):
                if (min_bound is None) or \
                        (get_12edo(scale, edo, smoothed_chord[0] - len(scale)) >= min_bound):
                    smoothed_chord[0] -= len(scale)
            else:
                if (max_bound is None) or \
                        (get_12edo(scale, edo, max(smoothed_chord[1:]) + len(scale)) <= max_bound):
                    smoothed_chord = [p + len(scale) for p in smoothed_chord]
                    smoothed_chord[0] -= len(scale)
        if max_bound is not None and\
                (get_12edo(scale, edo, max(smoothed_chord)) > max_bound):
            if (get_12edo(scale, edo, min(smoothed_chord) - len(scale)) > min_bound):
                smoothed_chord = [p - len(scale) for p in smoothed_chord]
        # todo: check the distance between notes
        smoothed_chords.append(smoothed_chord)
        prev_chord = smoothed_chord
    return smoothed_chords

def gen_chord_progression(scale, edo, tonic,
                          root_step=4, chord_step=2, chord_len=4, scheme=0,
                          prev_chord=None, min_bound=None, max_bound=None):
    roots = []
    root = tonic
    for i in range(beat_count):
        roots.append(root)
        root += root_step
    if scheme == 1:
        roots.reverse()
        roots = [roots[-1]] + roots[:-1]
    elif scheme == 2:
        roots.reverse()
    chords = []
    for root in roots:
        chord = []
        n = root
        for i in range(chord_len):
            chord.append(n)
            n += chord_step
        chords.append(chord)
    chords = smooth_chords(scale, edo, chords, start=prev_chord,
                           min_bound=min_bound, max_bound=max_bound)
    return chords
    # todo: add more schemes, e. g.:
    # i-i-ii-ii
    # i-i-i-ii
    # i-ii-i-ii
    # add slower changes, over several bars
    # ...


def gen_random_chord_progression(scale, edo, prev_chord=None, min_bound=None, max_bound=None):
    scheme = random.randint(0, 2)
    root_step = random.randint(1, len(scale) - 1)
    thirds = get_thirds(scale, edo)
    fourths = get_fourths(scale, edo)
    chord_step = random.choice(list(set(thirds + fourths)))
    chord_len = random.randint(3, 4)
#     modulate = random.choice([False, True])
    return gen_chord_progression(scale, edo, 0,
                                 root_step=root_step,
                                 chord_step=chord_step,
                                 chord_len=chord_len,
                                 scheme=scheme,
                                 prev_chord=prev_chord,
                                 min_bound=min_bound,
                                 max_bound=max_bound)

def gen_melody(scale, edo, chords, start=0, max_jump=None, max_rep=2, min_bound=None, max_bound=None,
               dont_dup=False):
    if max_jump is None:
        max_jump = random.choice([2, 3])
    rhythm = create_rhythm()
    total_count = beat_count * subbeat_count
    assert(len(rhythm) == total_count)
    assert(len(chords) == beat_count)
    note_idx = -1
    direction = 1
    last_note = start
    melody = []
    rep = 0
    for i in range(beat_count):
        had_prev_jump = False
        prev_jump = None
        for j in range(subbeat_count):
            if j == 0:
                available_notes = [p % len(scale) for p in chords[i]]
                if random.random() < 0.5:
                    direction *= -1
                if get_12edo(scale, edo, last_note) >= max_bound:
                    direction = -1
                if get_12edo(scale, edo, last_note) <= min_bound:
                    direction = 1
            else:
                available_notes = list(range(len(scale)))
            note_idx += 1
            if rhythm[note_idx] == 0:
                melody.append(['.'])
                continue
            if had_prev_jump and random.random() < 0.75:
                new_note = last_note + prev_jump
                last_note = new_note
            else:
                candidates = []
                jump = -1
                while len(candidates) <= 1 or jump <= max_jump:
                    jump += 1
                    new_note = last_note + jump * direction
                    if new_note % len(scale) in available_notes:
                        candidates.append(new_note)
                while True:
                    new_note = random.choice(candidates)
                    if new_note == last_note:
                        rep += 1
                        if rep > max_rep:
                            continue
                    else:
                        rep = 0
                        had_prev_jump = True
                        prev_jump = new_note - last_note
                        last_note = new_note
                        break
            melody.append([last_note])
    duplicate = False
    if not dont_dup and random.random() < 0.45:
        duplicate = True
    if duplicate:
        thirds = get_thirds(scale, edo)
        fourths = get_fourths(scale, edo)
        duplicate_step = random.choice(list(set(thirds + fourths)))
        duplicate_delay = random.choice([0, -1, 1])
        for idx in range(len(melody)):
            if melody[idx][0] == '.':
                continue
            if 0 <= idx + duplicate_delay < len(melody):
                melody[idx + duplicate_delay].append(melody[idx][0] + duplicate_step)
    return melody, last_note, duplicate


In [ ]:

dur = random.uniform(0.12, 0.3)
? note_count = 0
beat_count = random.randint(2, 5)
subbeat_count = random.randint(2, 5)

# edo = 22
scale = scales['orwell9']

prev_chord = [-len(scale)]
root = random.randint(0, len(scale) - 1)
start_note = root
start_note2 = root

counter = 0

last_edo_change = time.perf_counter()
edo_change_period = random.randint(60, 120)
last_instrument_switch = time.perf_counter()
last_tempo_switch = time.perf_counter()

while time.perf_counter() - start < 60:
    kicks = gen_random_percs(mode='kick')
    hihats = gen_random_percs(mode='hihat')
    snares = gen_random_percs(mode='snare')

    chords = gen_random_chord_progression(scale, edo, prev_chord=prev_chord,
                                          min_bound=-20, max_bound=-7)
    prev_chord = chords[-1]
    melody, last_note, is_dup = gen_melody(scale, edo, chords, start=start_note,
                                           min_bound=-4, max_bound=20)
    start_note = last_note
    melody2, last_note2, _ = gen_melody(scale, edo, chords, start=start_note2,
                                        min_bound=-16, max_bound=0,
                                        dont_dup=is_dup)
    start_note2 = last_note2
    chord_seq = gen_chord_seq(scale, edo, chords)
    print_cut('chords', chord_seq)
    print_cut('melody', melody)
    print_cut('melody2', melody2)
    switch_to_new_edo = False
    if time.perf_counter() - last_edo_change > edo_change_period:
        switch_to_new_edo = True
    modulate = False
    if not switch_to_new_edo:
        modulate = random.random() < 0.75
    reps = 2
    if not modulate and not switch_to_new_edo:
        reps = random.randint(2, 5)
    switch_instruments = (random.random() < 0.2) and (time.perf_counter() - last_instrument_switch > 10)
    if time.perf_counter() - last_instrument_switch > 30:
        switch_instruments = True
    switch_instruments_rep = random.randint(0, reps - 1)
    for j in range(reps):
        counter += 1
        mute_count = 0
        max_mute_count = 2
        cur_bass_amp = bass_amp
        cur_solo_amp = solo_amp
        cur_solo2_amp = solo2_amp
        if mute_count < max_mute_count:
            mute_bass = random.random() < 0.2
            if mute_bass:
                cur_bass_amp = 0
                mute_count += 1
        if mute_count < max_mute_count:
            mute_solo2 = random.random() < 0.2
            if mute_solo2:
                cur_solo2_amp = 0
                mute_count += 1
        if mute_count < max_mute_count:
            mute_solo = random.random() < 0.2
            if mute_solo:
                cur_solo_amp = 0
                mute_count += 1
        play_seqs(scale,
                  [kicks, hihats, snares, chord_seq, melody, melody2],
                  [kick, hihat, snare, bass, solo, solo2],
                  [random.uniform(0.7, 1.0), random.uniform(0.2, 0.4), random.uniform(0.2, 0.5),
                   cur_bass_amp, cur_solo_amp, cur_solo2_amp], dur=dur)
        if switch_instruments and j == switch_instruments_rep:
            last_instrument_switch = time.perf_counter()
            print('Changing instruments🎹🎸🎷')
            bass, bass_amp = get_random_bass()
            solo, solo_amp = get_random_solo()
            solo2, solo2_amp = get_random_solo2()
            switch_instruments = False
    switch_tempo = (not switch_instruments) and (random.random() < 0.2) and (time.perf_counter() - last_instrument_switch > 5)
    if time.perf_counter() - last_instrument_switch > 30:
        switch_tempo = True
    if switch_tempo:
        print('Changing tempo🥁')
        dur = random.uniform(0.12, 0.3)
        beat_count = random.randint(2, 5)
        subbeat_count = random.randint(2, 5)
    if modulate:
        print('modulating')
        new_edo = edo
        new_scale, new_root = modulate_and_recalc(scale, edo, chords[-1], root)
    elif switch_to_new_edo:
        edo_change_period = random.randint(60, 120)
        last_edo_change = time.perf_counter()
        new_edo = edo
        if len(all_edos) > 1:
            while new_edo == edo:
                new_edo = random.choice(all_edos)
        all_scales = gen_scales(new_edo)
        new_scale = random.choice(all_scales)
        new_root = random.randint(0, len(new_scale) - 1)
        printmd('# New EDO: ' + str(edo) + ' -> ' + str(new_edo))
    if modulate or switch_to_new_edo:
        start_note = approximate_note(new_scale, new_edo, start_note, scale, edo)
        start_note2 = approximate_note(new_scale, new_edo, start_note2, scale, edo)
        prev_chord_modulated = []
        for n in prev_chord:
            prev_chord_modulated.append(approximate_note(new_scale, edo, n, scale, edo))
        prev_chord = prev_chord_modulated
        edo = new_edo
        scale = new_scale
        root = new_root
        print('edo', edo)
        print('scale', scale)
printmd('# FIN!')